In [1]:
"""
This script test the performance of GA search by finding on average how many compuations (calling rkf_validator) required to 
find one of the top 10 feature subset and the feature subset
that is found by Grid Search (ground truth)

Acknowledgement:
This script was referenced from Lab 7
ChatGPT has been used for debugging and previded ideas for writing

"""

import numpy as np
import pandas as pd
from utilities import set_seed

DEVICE = 'cpu'

# set seed
SEED = 4660
if SEED != None:
    set_seed(SEED)

# Number of training for each hyperparameter combination
n_splits = 10
n_repeats = 4

# define GA settings
DNA_SIZE = 15             # number of bits in DNA
POP_SIZE = 100           # population size
CROSS_RATE = 0.8          # DNA crossover probability
MUTATION_RATE = 0.05      # mutation probability
   
TOURNAMENT_SIZE = 3       
N_ELITES = 5  # Number of elite individuals to carry forward

# The scores for all 4096 possible chromosomes
FITNESS_TABLE = pd.read_csv('results/feature_selection_fitness.csv', index_col=0, dtype={0: str})


def initialize_population():
    """
    Create the initial population
    output: a matrix (POP_SIZE x DNA_SIZE)
    """
    return np.random.randint(2, size=(POP_SIZE, DNA_SIZE))

def compute_fitness_all(population):
    """
    Compute the fitness of all individuals in a population
    return a fitness table containing fitness score for all individual in the population
    """
    chromosome_strings = [''.join(map(str, row)) for row in population] # convert into a list of string
    return FITNESS_TABLE.loc[chromosome_strings]


def tournament_selection(population, fitness_table):
    selected_indices = np.random.choice(len(population), TOURNAMENT_SIZE, replace=False)
    selected = population[selected_indices]
    fitnesses = np.array(fitness_table.iloc[selected_indices,0])
    best_index = np.argmin(fitnesses)
    return selected[best_index]

def crossover(parent1, parent2):
    """
    Perform single point crossover based on CROSS_RATE
    """
    if np.random.rand() < CROSS_RATE:
        point = np.random.randint(DNA_SIZE)
        child1 = np.hstack([parent1[:point], parent2[point:]])
        child2 = np.hstack([parent2[:point], parent1[point:]])
        return child1, child2
    else:
        return parent1.copy(), parent2.copy()

def mutate(child):
    """
    Mutate a child based on MUTATION_RATE
    """
    for point in range(DNA_SIZE):
        if np.random.rand() < MUTATION_RATE:
            child[point] = 1 if child[point] == 0 else 0
    return child

def select_elites(population, n_elites, fitness_table):
    """
    Get the top n_elites individual from the polupation
    """
    fitnesses = np.array(fitness_table.iloc[:,0])
    elite_indices = np.argsort(fitnesses)[:n_elites]
    return population[elite_indices]

def get_the_best_individual(fitness_table):
    """
    Get the best individual in a population given a individual-score mapping
    """
    best_individual = fitness_table.index[0]
    best_score = fitness_table.iloc[0,0]
    return best_individual, best_score

def genetic_algorithm_test(verbose = False):
    """
    Compute a population of hyperparameter combinations after N_GENERATIONS generations
    """

    top_10s = FITNESS_TABLE.sort_values(by='MSE', ascending=True).head(10)

    population = initialize_population() # population of the first generation

    num_gen_to_get_the_best = 0
    num_gen_to_get_top_10 = 0
    top_10_obtained = False

    # for generation in range(1, N_GENERATIONS + 1):

    generation = 1
    while(True):

        # Calculate the fitness for all individual after the i th generation
        fitness_table = compute_fitness_all(population)

        # Get the best chromosome and the score
        best_chromosome, best_score = get_the_best_individual(fitness_table)
        if verbose:
            print(f'Generation: {generation}   Best Feature Subset: {best_chromosome}   MSE: {best_score}')

        if not top_10_obtained and best_chromosome in top_10s.index:
            top_10_obtained = True
            num_gen_to_get_top_10 = generation
        
        if best_chromosome == top_10s.index[0]:
            num_gen_to_get_the_best = generation
            break

        # Start creating the (i + 1) th generation
        elites = select_elites(population, N_ELITES, fitness_table)

        new_population = elites.tolist()

        while len(new_population) < POP_SIZE:
            parent1 = tournament_selection(population, FITNESS_TABLE)
            parent2 = tournament_selection(population, FITNESS_TABLE)

            child1, child2 = crossover(parent1, parent2)

            child1 = mutate(child1)
            child2 = mutate(child2)

            new_population.extend([child1, child2])

        population = np.array(new_population[:POP_SIZE])
        generation += 1

    return num_gen_to_get_the_best, num_gen_to_get_top_10

In [2]:
# Run GA search 2000 times
NUM_TEST = 2000
result = []
for i in range(1, NUM_TEST + 1):
    num_gen_to_get_the_best, num_gen_to_get_top_10 = genetic_algorithm_test()
    result.append([num_gen_to_get_the_best, num_gen_to_get_top_10])
    if i % 20 == 0:
        print(f'{i}/{NUM_TEST}   Number of Gen to get the best chromosome: {num_gen_to_get_the_best}   Number of Gen to get one of the top 10s: {num_gen_to_get_top_10}')
result = np.array(result)

num_gen_to_get_the_best = result[:, 0]
num_gen_to_get_top_10 = result[:, 1]

20/2000   Number of Gen to get the best chromosome: 7   Number of Gen to get one of the top 10s: 7
40/2000   Number of Gen to get the best chromosome: 319   Number of Gen to get one of the top 10s: 4
60/2000   Number of Gen to get the best chromosome: 61   Number of Gen to get one of the top 10s: 15
80/2000   Number of Gen to get the best chromosome: 19   Number of Gen to get one of the top 10s: 19
100/2000   Number of Gen to get the best chromosome: 226   Number of Gen to get one of the top 10s: 22
120/2000   Number of Gen to get the best chromosome: 2   Number of Gen to get one of the top 10s: 2
140/2000   Number of Gen to get the best chromosome: 127   Number of Gen to get one of the top 10s: 11
160/2000   Number of Gen to get the best chromosome: 18   Number of Gen to get one of the top 10s: 16
180/2000   Number of Gen to get the best chromosome: 104   Number of Gen to get one of the top 10s: 10
200/2000   Number of Gen to get the best chromosome: 87   Number of Gen to get one of t

In [3]:
print()
print("Number of Gen to get the best chromosome:")
print(f'Mean: {np.mean(num_gen_to_get_the_best)}')
print(f'Median: {np.median(num_gen_to_get_the_best)}')
print(f'Standard Deviation: {np.std(num_gen_to_get_the_best)}')
print()
print("Number of Gen to get one of the top 10s:")
print(f'Mean: {np.mean(num_gen_to_get_top_10)}')
print(f'Median: {np.median(num_gen_to_get_top_10)}')
print(f'Standard Deviation: {np.std(num_gen_to_get_top_10)}')


Number of Gen to get the best chromosome:
Mean: 211.791
Median: 153.0
Standard Deviation: 205.88213695947496

Number of Gen to get one of the top 10s:
Mean: 23.176
Median: 19.0
Standard Deviation: 17.624699259845542
